### 1 *Importing Necessary Libraries*

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import GeoJson
from folium.plugins import MarkerCluster
import geopandas as gpd

### 2 *Loading Dataset and Basic Preprocessing*

In [2]:
data=pd.read_csv("crimes_dataset.csv")
data.columns = data.columns.str.lower().str.replace(' ', '_')
data['block_location']=data['block_location'].str.replace('\n', ' ', regex=True)

In [3]:
data.head(3)

,id,caseno,offense,cvlegend,cvdow,block_location,blkaddr,city,state,day,lat,lon,timestamp
0,0.0,17091420.0,BURGLARY AUTO,BURGLARY - VEHICLE,0.0,"2500 LE CONTE AVE Berkeley, CA (37.876965, -12...",2500 LE CONTE AVE,Berkeley,CA,Sunday,37.876965,-122.260544,23/07/2017 6:00
1,1.0,17020462.0,THEFT FROM PERSON,LARCENY,4.0,"2200 SHATTUCK AVE Berkeley, CA (37.869363, -12...",2200 SHATTUCK AVE,Berkeley,CA,Thursday,37.869363,-122.268028,13/04/2017 8:45
2,2.0,17050275.0,BURGLARY AUTO,BURGLARY - VEHICLE,4.0,"200 UNIVERSITY AVE Berkeley, CA (37.865491, -1...",200 UNIVERSITY AVE,Berkeley,CA,Thursday,37.865491,-122.310065,24/08/2017 18:30


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5508 entries, 0 to 5507
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2000 non-null   float64
 1   caseno          2000 non-null   float64
 2   offense         2000 non-null   object 
 3   cvlegend        2000 non-null   object 
 4   cvdow           2000 non-null   float64
 5   block_location  2000 non-null   object 
 6   blkaddr         1992 non-null   object 
 7   city            2000 non-null   object 
 8   state           2000 non-null   object 
 9   day             2000 non-null   object 
 10  lat             1908 non-null   float64
 11  lon             1908 non-null   float64
 12  timestamp       2000 non-null   object 
dtypes: float64(5), object(8)
memory usage: 559.5+ KB


In [5]:
data.describe(include="all")

,id,caseno,offense,cvlegend,cvdow,block_location,blkaddr,city,state,day,lat,lon,timestamp
count,2000.000000,2.000000e+03,2000,2000,2000.000000,2000,1992,2000,2000,2000,1908.000000,1908.000000,2000
unique,NaN,NaN,28,21,NaN,1013,1012,1,1,7,NaN,NaN,1905
top,NaN,NaN,BURGLARY AUTO,LARCENY,NaN,"2100 SHATTUCK AVE Berkeley, CA (37.871167, -12...",2100 SHATTUCK AVE,Berkeley,CA,Friday,NaN,NaN,21/08/2017 17:00
freq,NaN,NaN,379,427,NaN,36,36,2000,2000,315,NaN,NaN,4
mean,999.500000,1.704434e+07,NaN,NaN,3.094000,NaN,NaN,NaN,NaN,NaN,37.868259,-122.275695,NaN
std,577.494589,2.674710e+04,NaN,NaN,1.971321,NaN,NaN,NaN,NaN,NaN,0.010352,0.015878,NaN
min,0.000000,1.700023e+07,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,37.821533,-122.317566,NaN
25%,499.750000,1.702349e+07,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,37.861991,-122.288543,NaN
50%,999.500000,1.703707e+07,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,37.868164,-122.270706,NaN
75%,1499.250000,1.704907e+07,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,37.872989,-122.265050,NaN


### Q1 *a*

In [6]:
print(f"Most Frequent Offense is \"{data['offense'].value_counts().idxmax()}\"")
print(f"With the frequency of {data['offense'].value_counts().max()}")

Most Frequent Offense is "BURGLARY AUTO"
With the frequency of 379


### Q1 b
**(CVDOW is the Label Encoded Format of DAY, using DAY instead of CVDOW)**

In [7]:
data.groupby('day')['offense'].count()

day
Friday       315
Monday       287
Saturday     287
Sunday       245
Thursday     298
Tuesday      292
Wednesday    276
Name: offense, dtype: int64

In [8]:
print(f"Day with most crimes is \"{data.groupby('day')['offense'].count().idxmax()}\"")
print(f"With the frequency of {data.groupby('day')['offense'].count().max()}")

Day with most crimes is "Friday"
With the frequency of 315


### Q1 c

In [9]:
print(f"Location with most crimes is \"{data.groupby(['lat', 'lon'])['offense'].count().idxmax()}\"")
print(f"With the frequency of {data.groupby(['lat', 'lon'])['offense'].count().max()}")

Location with most crimes is "(37.871167, -122.268285)"
With the frequency of 40


### Q1 d

In [10]:
data['block_location'].value_counts()

2100 SHATTUCK AVE Berkeley, CA (37.871167, -122.268285)                   36
2200 SHATTUCK AVE Berkeley, CA (37.869363, -122.268028)                   20
2300 TELEGRAPH AVE Berkeley, CA (37.868714, -122.259189)                  20
200 UNIVERSITY AVE Berkeley, CA (37.865491, -122.310065)                  17
1900 SHATTUCK AVE Berkeley, CA (37.873687, -122.268616)                   17
                                                                          ..
BANCROFT WAY &amp; ELLSWORTH ST Berkeley, CA (37.86814, -122.263736)       1
1100 OXFORD ST Berkeley, CA (37.88878, -122.267659)                        1
FULTON STREET &amp; KITTREDGE ST Berkeley, CA (37.868706, -122.266279)     1
CHANNING WAY &amp; WARRING ST Berkeley, CA (37.867919, -122.250841)        1
2700 STUART ST Berkeley, CA (37.859846, -122.253325)                       1
Name: block_location, Length: 1013, dtype: int64

In [11]:
print(f"Blocks with most crimes is \"{data['block_location'].value_counts().idxmax()}\"")
print(f"With the frequency of {data['block_location'].value_counts().max()}")

Blocks with most crimes is "2100 SHATTUCK AVE Berkeley, CA (37.871167, -122.268285)"
With the frequency of 36


## Q2 Showing the Frequency Map
Note: This data is plotted based on dropping nulls, it was a part of Q1, but was not carried there

In [14]:
m = folium.Map(location=[data['lat'].mean(), data['lon'].mean()], zoom_start=13)
marker_cluster = MarkerCluster().add_to(m)



for i, j in data.iterrows():
    if pd.notna(j['lat']) and pd.notna(j['lon']):
        t=f"Offense: {j['offense']}"
        folium.Marker([j['lat'], j['lon']], t).add_to(marker_cluster)

display(m)